# Example Sample Hotel and Flight Booker Agent 

This solution will help you book flight tickets and hotel.  The scenario is a trip London Heathrow LHR Feb 20th 2024 to New York JFK returning Feb 27th 2025 flying economy with British Airways only. I want a stay in a Hilton hotel in New York please provide costs for the flight and hotel.

# Initialize the Azure AI Agent Service and get configuration information from **.env**

### **.env** 

Create a .env file 

**.env** contains the connection string of Azure AI Agent Service, the model used by AOAI, and the corresponding Google API Search service API, ENDPOINT, etc.

- **AZURE_AI_AGENT_MODEL_DEPLOYMENT_NAME** = "Your Azure AI Agent Service Model Deployment Name"

[**NOTE**] You will need a model with 100,000 Rate Limit (Tokens per minute)  Rate Limit of 600 (Request per minute)

  You can get model in Azure AI Foundry - Model and Endpoint. 


- **AZURE_AI_AGENT_PROJECT_CONNECTION_STRING** = "Your Azure AI Agent Service Project Connection String"

  You can get the project connection string in your project overview in  AI ​​Foundry Portal Screen.

- **SERPAPI_SEARCH_API_KEY** = "Your SERPAPI Search API KEY"

To get the Model Deployment Name and Project Connection String of Azure AI Agent Service, you need to create Azure AI Agent Service. It is recommended to use [this template](https://portal.azure.com/#create/Microsoft.Template/uri/https%3A%2F%2Fraw.githubusercontent.com%2Ffosteramanda%2Fazure-agent-quickstart-templates%2Frefs%2Fheads%2Fmaster%2Fquickstarts%2Fmicrosoft.azure-ai-agent-service%2Fstandard-agent%2Fazuredeploy.json) to create it directly （***Note:*** Azure AI Agent Service is currently set in a limited region. It is recommended that you refer to [this link](https://learn.microsoft.com/en-us/azure/ai-services/agents/concepts/model-region-support) to set the region)

Agent needs to access SERPAPI. It is recommended to register using [this link](https://serpapi.com/searches). After registration, you can obtain a unique API KEY and ENDPOINT

# Setup 

To run this notebook, you will need to make sure you've installed the required libraries by running `pip install -r requirements.txt`.

In [6]:
from semantic_kernel import __version__
import json
import os

from dotenv import load_dotenv

from IPython.display import display, HTML

from typing import Annotated
from openai import AsyncOpenAI

from semantic_kernel.agents import ChatCompletionAgent, ChatHistoryAgentThread
from semantic_kernel.connectors.ai.open_ai import OpenAIChatCompletion
from semantic_kernel.contents import FunctionCallContent, FunctionResultContent, StreamingTextContent
from semantic_kernel.functions import kernel_function

__version__

'1.37.0'

Your Semantic Kernel version should be at least 1.27.2.

Load your .env file setting and resources please ensure you have added your keys and setting and created a local .env file.

In [7]:
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

True

# Log in to Azure

You now need to log in to Azure. Open a terminal and run the following command:

```bash
az login
```

This command will prompt you to enter your Azure credentials, enabling the Azure AI Agent service to function correctly.

# Explanation:
This is a variable that stores the API key for accessing a SERP (Search Engine Results Page) API service. An API key is a unique identifier used to authenticate requests associated with your account.

Purpose: The purpose of this line is to store the API key in a variable so that it can be used to authenticate requests to the SERP API service. The API key is required to access the service and perform searches.
How to Get a SERP API Key: To get a SERP API key, follow these general steps at https://serpapi.com (the exact steps may vary depending on the specific SERP API service you are using):

Choose a SERP API Service: There are several SERP API services available, such as SerpAPI, Google Custom Search JSON API, and others. Choose the one that best fits your needs.

Sign Up for an Account: Go to the website of the chosen SERP API service and sign up for an account. You may need to provide some basic information and verify your email address.

Create an API Key: After signing up, log in to your account and navigate to the API section or dashboard. Look for an option to create or generate a new API key.
Copy the API Key to your .env file.

In [8]:
SERP_API_KEY=os.getenv("SERPAPI_SEARCH_API_KEY")

# Explanation:
BASE_URL: This is a variable that stores the base URL for the SERP API endpoint. The variable name BASE_URL is a convention used to indicate that this URL is the starting point for making API requests.
'https://serpapi.com/search':

This is the actual URL string assigned to the BASE_URL variable. It represents the endpoint for performing search queries using the SERP API.

# Purpose:
The purpose of this line is to define a constant that holds the base URL for the SERP API. This URL will be used as the starting point for constructing API requests to perform search operations.

# Usage:
By defining the base URL in a variable, you can easily reuse it throughout your code whenever you need to make requests to the SERP API. This makes your code more maintainable and reduces the risk of errors from hardcoding the URL in multiple places. The current example is https://serpapi.com/search?engine=bing which is using Bing search API. Different API can be selected at https://Serpapi.com

In [17]:
BASE_URL = 'https://serpapi.com/search'

# Explanation:

This is where your plugin code is located.

Class Definition: `class BookingPlugin`: Defines a class named BookingPlugin that contains methods for booking hotels and flights.

Hotel Booking Method:

- `@kernel_function(description="booking hotel")`: A decorator that describes the function as a kernel function for booking hotels.
- `def booking_hotel(self, query: Annotated[str, "The name of the city"], check_in_date: Annotated[str, "Hotel Check-in Time"], check_out_date: Annotated[str, "Hotel Check-out Time"]) -> Annotated[str, "Return the result of booking hotel information"]:`: Defines a method for booking hotels with annotated parameters and return type.

The method constructs a dictionary of parameters for the hotel booking request and sends a GET request to the SERP API. It checks the response status and returns the hotel properties if successful, or None if the request failed.

Flight Booking Method: 

- `@kernel_function(description="booking flight")`: A decorator that describes the function as a kernel function for booking flights.
- `def booking_flight(self, origin: Annotated[str, "The name of Departure"], destination: Annotated[str, "The name of Destination"], outbound_date: Annotated[str, "The date of outbound"], return_date: Annotated[str, "The date of Return_date"]) -> Annotated[str, "Return the result of booking flight information"]:`: Defines a method for booking flights with annotated parameters and return type.

The method constructs dictionaries of parameters for the outbound and return flight requests and sends GET requests to the SERP API. It checks the response status and appends the flight information to the result string if successful, or prints an error message if the request failed. The method returns the result string containing the flight information.


In [ ]:
import requests

from typing import Annotated

from semantic_kernel.functions import kernel_function

# Define Booking Plugin
class BookingPlugin:
    """Booking Plugin for customers"""

    @kernel_function(description="booking hotel")
    def booking_hotel(
        self, 
        query: Annotated[str, "The name of the city"], 
        check_in_date: Annotated[str, "Hotel Check-in Time"], 
        check_out_date: Annotated[str, "Hotel Check-out Time"],
    ) -> Annotated[str, "Return the result of booking hotel information"]:
        """
        Function to book a hotel.
        Parameters:
        - query: The name of the city
        - check_in_date: Hotel Check-in Time
        - check_out_date: Hotel Check-out Time
        Returns:
        - The result of booking hotel information
        """

        # Define the parameters for the hotel booking request
        params = {
            "engine": "google_hotels",
            "q": query,
            "check_in_date": check_in_date,
            "check_out_date": check_out_date,
            "adults": "1",
            "currency": "GBP",
            "gl": "uk",
            "hl": "en",
            "api_key": SERP_API_KEY
        }

        # Send the GET request to the SERP API
        response = requests.get(BASE_URL, params=params)

        # Check if the request was successful
        if response.status_code == 200:
            # Parse the response content as JSON
            response = response.json()
            # Return the properties from the response
            return response["properties"]
        else:
            # Return None if the request failed
            return None

    @kernel_function(description="convert Airport Name to IATA code")
    def convert_airport_name_to_iata_code(
        self,
        airport_name: Annotated[str, "The name or description of the airport"]
    ) -> Annotated[str, "Return the 3-letter IATA airport code"]:
        """
        Function to convert airport name to IATA code.
        Parameters:
        - airport_name: The name or description of the airport
        Returns:
        - The 3-letter IATA airport code
        """
        
        # Comprehensive airport code mapping
        airport_codes = {
            # London airports
            "london heathrow": "LHR",
            "heathrow": "LHR", 
            "london heathrow lhr": "LHR",
            "lhr": "LHR",
            "london gatwick": "LGW",
            "gatwick": "LGW",
            "london stansted": "STN",
            "stansted": "STN",
            "london luton": "LTN",
            "luton": "LTN",
            "london city": "LCY",
            
            # New York airports
            "new york jfk": "JFK",
            "john f kennedy": "JFK",
            "kennedy": "JFK",
            "jfk": "JFK",
            "new york": "JFK",  # Default to JFK for "New York"
            "new york laguardia": "LGA",
            "laguardia": "LGA",
            "lga": "LGA",
            "new york newark": "EWR",
            "newark": "EWR",
            "ewr": "EWR",
            
            # Other major airports
            "los angeles": "LAX",
            "lax": "LAX",
            "chicago ohare": "ORD",
            "ohare": "ORD",
            "ord": "ORD",
            "miami": "MIA",
            "mia": "MIA",
            "paris charles de gaulle": "CDG",
            "charles de gaulle": "CDG",
            "cdg": "CDG",
            "paris orly": "ORY",
            "orly": "ORY",
            "amsterdam": "AMS",
            "schiphol": "AMS",
            "ams": "AMS",
            "frankfurt": "FRA",
            "fra": "FRA",
            "tokyo narita": "NRT",
            "narita": "NRT",
            "nrt": "NRT",
            "tokyo haneda": "HND",
            "haneda": "HND",
            "hnd": "HND",
            "sydney": "SYD",
            "syd": "SYD",
            "melbourne": "MEL",
            "mel": "MEL",
            "dubai": "DXB",
            "dxb": "DXB",
            "singapore": "SIN",
            "changi": "SIN",
            "sin": "SIN"
        }
        
        # Clean and normalize the input
        normalized_name = airport_name.lower().strip()
        
        # Direct lookup
        if normalized_name in airport_codes:
            return airport_codes[normalized_name]
        
        # Check if it's already a 3-letter code
        if len(normalized_name) == 3 and normalized_name.isalpha():
            return normalized_name.upper()
        
        # Partial matching for more flexibility
        for key, code in airport_codes.items():
            if key in normalized_name or normalized_name in key:
                return code
        
        # If no match found, try to extract 3-letter code from the input
        words = normalized_name.split()
        for word in words:
            if len(word) == 3 and word.isalpha():
                return word.upper()
        
        # Return the first 3 characters as uppercase if nothing else works
        return normalized_name[:3].upper() if len(normalized_name) >= 3 else normalized_name.upper()

    @kernel_function(description="booking flight")
    def booking_flight(
        self, 
        origin: Annotated[str, "The name of Departure"], 
        destination: Annotated[str, "The name of Destination"], 
        outbound_date: Annotated[str, "The date of outbound"], 
        return_date: Annotated[str, "The date of Return_date"],
    ) -> Annotated[str, "Return the result of booking flight information"]:
        """
        Function to book a flight.
        Parameters:
        - origin: The name of Departure
        - destination: The name of Destination
        - outbound_date: The date of outbound
        - return_date: The date of Return_date
        Returns:
        - The result of booking flight information
        """
        
        # Convert airport names to IATA codes using the helper function
        origin_code = self.convert_airport_name_to_iata_code(origin)
        destination_code = self.convert_airport_name_to_iata_code(destination)
        
        print(f"Converted {origin} -> {origin_code}")
        print(f"Converted {destination} -> {destination_code}")


        # Use Google Flights specific URL
        FLIGHT_BASE_URL = 'https://serpapi.com/search'
        
        # Define the parameters for the outbound flight request
        go_params = {
            "engine": "google_flights",
            "departure_id": origin_code,  # Fixed: use actual variable instead of string
            "arrival_id": destination_code,  # Fixed: use actual variable instead of string
            "outbound_date": outbound_date,  # Fixed: use actual variable instead of string
            "return_date": return_date,  # Fixed: use actual variable instead of string
            "currency": "GBP",
            "hl": "en",
            "api_key": SERP_API_KEY  # Fixed: use actual variable instead of string
        }

        print(go_params)

        # Send the GET request for the outbound flight
        go_response = requests.get(FLIGHT_BASE_URL, params=go_params)

        # Initialize the result string
        result = ''

        # Check if the outbound flight request was successful
        if go_response.status_code == 200:
            response = go_response.json()
            # Extract only essential flight information instead of entire response
            if 'best_flights' in response:
                result += "# outbound flights:\n"
                for flight in response['best_flights'][:3]:  # Limit to top 3 flights
                    result += f"Flight: {flight.get('flights', [{}])[0].get('flight_number', 'N/A')}, "
                    result += f"Price: {flight.get('price', 'N/A')}, "
                    result += f"Duration: {flight.get('total_duration', 'N/A')}\n"
            else:
                result += "# outbound flights: No flights found\n"
        else:
            # Print an error message if the request failed
            print(f'Outbound flight error: {go_response.status_code}')
            print(f'Error response: {go_response.text}')

        # Define the parameters for the return flight request
        back_params = {
            "engine": "google_flights",  # Fixed: add required engine parameter
            "departure_id": destination_code,
            "arrival_id": origin_code,
            "outbound_date": outbound_date,
            "return_date": return_date,
            "currency": "GBP",
            "hl": "en",
            "api_key": SERP_API_KEY
        }

        # Send the GET request for the return flight
        back_response = requests.get(FLIGHT_BASE_URL, params=back_params)

        # Check if the return flight request was successful
        if back_response.status_code == 200:
            # Parse the response content as JSON
            response = back_response.json()
            # Append the return flight information to the result
            result += "\n # return \n" + str(response)
        else:
            print(f"Return flight request failed with status code: {back_response.status_code}")
            print(f"Error response: {back_response.text}")

        # Print the result
        print(result)

        # Return the result
        return result


# Explanation:
Import Statements: Import necessary modules for Azure credentials, AI agent, chat message content, author role, and kernel function decorator.

Asynchronous Context Manager: async with (DefaultAzureCredential() as creds, AzureAIAgent.create_client(credential=creds, conn_str="...") as client,): This sets up an asynchronous context manager to handle Azure credentials and create an AI agent client.

Agent Name and Instructions: 
- `AGENT_NAME = "BookingAgent"`: Defines the name of the agent.
- `AGENT_INSTRUCTIONS = """..."""`: Provides detailed instructions for the agent on how to handle booking requests.

Create Agent Definition: `agent_definition = await client.agents.create_agent(...)`: Creates an agent definition with the specified model, name, and instructions.

Create AzureAI Agent: `agent = AzureAIAgent(...)`: Creates an AzureAI agent using the client, agent definition, and the defined plugin.

Create Thread: `thread: AzureAIAgentThread | None = None`: Create a thread for the agent. It isn't required to first create a thread - if the value of `None` is provided, a new thread will be created during the first invocation and returned as part of the response.

User Inputs: `user_inputs = ["..."]`: Defines a list of user inputs for the agent to process.

In the finally block, delete the thread and agent to clean up resources.

# Authentication

The `DefaultAzureCredential` class is part of the Azure SDK for Python. It provides a default way to authenticate with Azure services. It attempts to authenticate using multiple methods in a specific order, such as environment variables, managed identity, and Azure CLI credentials.

Asynchronous Operations: The aio module indicates that the DefaultAzureCredential class supports asynchronous operations. This means you can use it with asyncio to perform non-blocking authentication requests.

In [32]:
# Import necessary modules
from azure.identity.aio import DefaultAzureCredential
from semantic_kernel.agents import AzureAIAgent, AzureAIAgentSettings, AzureAIAgentThread
from openai import AsyncOpenAI

# Fix deprecated create method
ai_agent_settings = AzureAIAgentSettings()

# Azure AI Setting
client = AsyncOpenAI(
    api_key=os.getenv("GITHUB_TOKEN"), 
    base_url="https://models.inference.ai.azure.com/",
)

# Define the agent's name and instructions
AGENT_NAME = "BookingAgent"
AGENT_INSTRUCTIONS = """
You are a booking agent, help me to book flights or hotels.

Thought: Understand the user's intention and confirm whether to use the reservation system to complete the task.

Action:
- If booking a flight via functions calling, convert the departure_id (departure name) and arrival_id (destination name) into airport codes (3-letter IATA codes like LHR for London Heathrow, JFK for New York JFK).
- If booking a hotel or flight, use the corresponding API to call. Ensure that the necessary parameters are available. If any parameters are missing, use default values or assumptions to proceed.
- If it is not a hotel or flight booking, respond with the final answer only.
- Output the results using a markdown table:
- For flight bookings, separate the outbound and return contents and list them in the order of Departure_airport Name | Airline | Flight Number | Departure Time | Arrival_airport Name | Arrival Time | Duration | Airplane | Travel Class | Price (USD) | Legroom | Extensions | Carbon Emissions (kg).
- For hotel bookings, list them in the order of Properties Name | Properties description | check_in_time | check_out_time | prices | nearby_places | hotel_class | gps_coordinates.

"""

chat_completion_service = OpenAIChatCompletion(
    ai_model_id="gpt-4.1",
    async_client=client,
)

# Create the agent
agent = ChatCompletionAgent(
    service=chat_completion_service,
    name=AGENT_NAME,
    instructions=AGENT_INSTRUCTIONS,
    plugins=[BookingPlugin()],
)


# Create a new thread for the agent
# If no thread is provided, a new thread will be
# created and returned with the initial response
thread: AzureAIAgentThread | None = None

# This is your prompt for the activity or task you want to complete 
# Define user inputs for the agent to process we have provided some example prompts to test and validate 
user_inputs = [
    # "Can you tell me the round-trip air ticket from  London to New York JFK aiport, the departure time is February 17, 2025, and the return time is February 23, 2025"
    # "Book a hotel in New York from Feb 20,2025 to Feb 24,2025"
    "Help me book flight tickets and hotel for the following trip London Heathrow LHR Feb 20th 2026 to New York JFK returning Feb 27th 2026 flying economy with British Airways only. I want a stay in a Hilton hotel in New York please provide costs for the flight and hotel"
    # "I have a business trip from London LHR to New York JFK on Feb 20th 2025 to Feb 27th 2025, can you help me to book a hotel and flight tickets"
]

try:
    # Process each user input
    for user_input in user_inputs:
        print(f"# User: '{user_input}'")
        # Get the agent's response for the specified thread
        response = await agent.get_response(
            messages=user_input,
            thread=thread,
        )
        thread = response.thread
        # Print the agent's response
        print(f"{response.name}: '{response.content}'")
finally:
    # Clean up by deleting the thread and agent
    await thread.delete() if thread else None


# User: 'Help me book flight tickets and hotel for the following trip London Heathrow LHR Feb 20th 2026 to New York JFK returning Feb 27th 2026 flying economy with British Airways only. I want a stay in a Hilton hotel in New York please provide costs for the flight and hotel'
Converted London Heathrow -> LHR
Converted New York JFK -> JFK
{'engine': 'google_flights', 'departure_id': 'LHR', 'arrival_id': 'JFK', 'outbound_date': '2026-02-20', 'return_date': '2026-02-27', 'currency': 'GBP', 'hl': 'en', 'api_key': '996986f00b8b10cf394a99afc0691189097bba564d4b82dce970a5d01764a350'}
# outbound 
 {'search_metadata': {'id': '68ca6da0ef15ced078cfbf0f', 'status': 'Success', 'json_endpoint': 'https://serpapi.com/searches/49759a7b8252eac6/68ca6da0ef15ced078cfbf0f.json', 'created_at': '2025-09-17 08:13:20 UTC', 'processed_at': '2025-09-17 08:13:20 UTC', 'google_flights_url': 'https://www.google.com/travel/flights?hl=en&gl=us&curr=GBP&tfs=CBwQAhoeEgoyMDI2LTAyLTIwagcIARIDTEhScgcIARIDSkZLGh4SCjIwMjYtMD

ServiceResponseException: ("<class 'semantic_kernel.connectors.ai.open_ai.services.open_ai_chat_completion.OpenAIChatCompletion'> service failed to complete the prompt", APIStatusError("Error code: 413 - {'error': {'code': 'tokens_limit_reached', 'message': 'Request body too large for gpt-4.1 model. Max size: 16000 tokens.', 'details': 'Request body too large for gpt-4.1 model. Max size: 16000 tokens.'}}"))